# Modelo de rede neural

Utilizando a base v5.

## Preparando o ambiente

In [168]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import time
import numpy as np
import matplotlib.pyplot as plt
import pylab as plot
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.callbacks import EarlyStopping
params = { 
    'axes.labelsize': "large",
    'xtick.labelsize': 'x-large',
    'legend.fontsize': 20,
    'figure.dpi': 150,
    'figure.figsize': [10, 6]
}
plot.rcParams.update(params)

In [2]:
SEED = 1
np.random.seed(SEED)

## Carregando os dados

In [69]:
treino = pd.read_csv('https://raw.githubusercontent.com/SalatielBairros/kaggle-titanic/main/data/processed_v5_1/train.csv')
teste = pd.read_csv('https://raw.githubusercontent.com/SalatielBairros/kaggle-titanic/main/data/processed_v5_1/test.csv')

X_treino = treino.drop(columns=['PassengerId','Survived', 'Fare'])
y_treino = treino['Survived']
X_teste = teste.drop(columns=['PassengerId', 'Fare'])
validation_passanger_ids = teste['PassengerId']

treino.head()

,Title_Mr,Sex,Age,Title_Mrs,Fare,Pclass_3,Cabin_T,SmallFamily,Title_Miss,Pclass_1,...,Ticket_C,Cabin_A,Cabin_F,Title_Royalty,Ticket_7,Ticket_F,Ticket_6,Ticket_4,PassengerId,Survived
0,1,1,0.2750,0,0.014151,1,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0.4750,1,0.139136,0,0,1,0,1,...,0,0,0,0,0,0,0,0,2,1
2,0,0,0.3250,0,0.015469,1,1,0,1,0,...,0,0,0,0,0,0,0,0,3,1
3,0,0,0.4375,1,0.103644,0,0,1,0,1,...,0,0,0,0,0,0,0,0,4,1
4,1,1,0.4375,0,0.015713,1,1,0,0,0,...,0,0,0,0,0,0,0,0,5,0


In [123]:
X_treino.columns

Index(['Title_Mr', 'Sex', 'Age', 'Title_Mrs', 'Pclass_3', 'Cabin_T',
       'SmallFamily', 'Title_Miss', 'Pclass_1', 'FamilySize', 'LargeFamily',
       'Ticket_1', 'SibSp', 'Singleton', 'Ticket_3', 'Pclass_2',
       'Title_Master', 'Title_Officer', 'Parch', 'Embarked_C', 'Ticket_2',
       'Cabin_E', 'Embarked_S', 'Cabin_D', 'Embarked_Q', 'Cabin_C', 'Ticket_S',
       'Cabin_B', 'Ticket_P', 'Ticket_A', 'Ticket_C', 'Cabin_A', 'Cabin_F',
       'Title_Royalty', 'Ticket_7', 'Ticket_F', 'Ticket_6', 'Ticket_4'],
      dtype='object')

In [176]:
# keep = ['Title_Mr', 'Sex', 'Age', 'Title_Mrs', 'Pclass_3', 'Cabin_T',
#        'SmallFamily', 'Title_Miss', 'Pclass_1', 'FamilySize', 'LargeFamily',
#        'Ticket_1', 'SibSp', 'Singleton', 'Ticket_3', 'Pclass_2',
#        'Title_Master', 'Title_Officer', 'Parch', 'Embarked_C', 'Ticket_2',
#        'Embarked_S','Cabin_B', 'Ticket_P']
keep = X_treino.columns
new_x_treino = X_treino[keep]

In [192]:
classifier = Sequential()
classifier.add(Dense(activation="relu", input_dim=38, units=16, kernel_initializer="GlorotUniform"))
classifier.add(Dense(activation="relu", units=24, kernel_initializer="GlorotUniform", use_bias=True))
classifier.add(Dense(activation="relu", units=16, kernel_initializer="GlorotUniform"))
classifier.add(Dense(activation="relu", units=16, kernel_initializer="GlorotUniform", use_bias=True))
classifier.add(Dropout(0.5))
classifier.add(Dense(activation="sigmoid", units=8, kernel_initializer="GlorotUniform"))
classifier.add(Dense(activation="relu", units=8, kernel_initializer="GlorotUniform"))
classifier.add(Dropout(0.5))
classifier.add(Dense(activation="sigmoid", units=4, kernel_initializer="GlorotUniform"))
classifier.add(Dense(activation="sigmoid", units=1, kernel_initializer="GlorotUniform"))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.summary()

Model: "sequential_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_162 (Dense)           (None, 16)                624       
                                                                 
 dense_163 (Dense)           (None, 24)                408       
                                                                 
 dense_164 (Dense)           (None, 16)                400       
                                                                 
 dense_165 (Dense)           (None, 16)                272       
                                                                 
 dropout_48 (Dropout)        (None, 16)                0         
                                                                 
 dense_166 (Dense)           (None, 8)                 136       
                                                                 
 dense_167 (Dense)           (None, 8)               

In [188]:
target = y_treino.values
features = new_x_treino.values

x_teste_features = X_teste[keep].values

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

In [193]:
history = classifier.fit(features, target, batch_size = 10, epochs  = 200, validation_split=0.2, verbose = 1, shuffle=True, callbacks=[es])

Epoch 1/200
72/72 [==============================] - 1s 3ms/step - loss: 0.7277 - accuracy: 0.3933 - val_loss: 0.7011 - val_accuracy: 0.3575
Epoch 2/200
72/72 [==============================] - 0s 2ms/step - loss: 0.6922 - accuracy: 0.5070 - val_loss: 0.6702 - val_accuracy: 0.6425
Epoch 3/200
72/72 [==============================] - 0s 2ms/step - loss: 0.6695 - accuracy: 0.6011 - val_loss: 0.6376 - val_accuracy: 0.6425
Epoch 4/200
72/72 [==============================] - 0s 1ms/step - loss: 0.6376 - accuracy: 0.6728 - val_loss: 0.5760 - val_accuracy: 0.7877
Epoch 5/200
72/72 [==============================] - 0s 1ms/step - loss: 0.5978 - accuracy: 0.7261 - val_loss: 0.5138 - val_accuracy: 0.8212
Epoch 6/200
72/72 [==============================] - 0s 1ms/step - loss: 0.5852 - accuracy: 0.7331 - val_loss: 0.4944 - val_accuracy: 0.8380
Epoch 7/200
72/72 [==============================] - 0s 1ms/step - loss: 0.5365 - accuracy: 0.7739 - val_loss: 0.4693 - val_accuracy: 0.8547
Epoch 8/200
7

In [194]:
Y_pred = classifier.predict(x_teste_features).round().astype(int)

In [195]:
solution = pd.DataFrame(Y_pred, validation_passanger_ids, columns = ['Survived'])
predictions = solution['Survived'].values

In [196]:
gt = pd.read_csv('../../data/original/ground_truth.csv')
print(classification_report(gt['Survived'], predictions))
print(accuracy_score(gt['Survived'], predictions))

              precision    recall  f1-score   support

           0       0.81      0.82      0.81       260
           1       0.70      0.68      0.69       158

    accuracy                           0.77       418
   macro avg       0.75      0.75      0.75       418
weighted avg       0.77      0.77      0.77       418

0.7679425837320574


In [187]:
classifier.save('../../data/nn_models/modelo_equivalente_nn.h5')
classifier.save_weights('../../data/nn_models/w_modelo_equivalente_nn.h5')


In [203]:
####################################
#  Libraries
####################################

import numpy as np 
import pandas as pd 
# Data processing, metrics and modeling
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import cross_val_score, StratifiedKFold
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout
# Reproductibility
from numpy.random import seed
seed(1002)
import tensorflow 

tensorflow.random.set_seed(1002)
# import set_random_seed
# set_random_seed(1002)

####################################
# Importing data and merging
####################################

# Reading dataset
train = pd.read_csv("https://raw.githubusercontent.com/SalatielBairros/kaggle-titanic/main/data/original/train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/SalatielBairros/kaggle-titanic/main/data/original/test.csv")

# Adding a column in each dataset before merging
train['Type'] = 'train'
test['Type'] = 'test'

# Merging train and test
data = train.append(test)

####################################
# Missing values and new features
####################################

# Title
data['Title'] = data['Name']

# Cleaning name and extracting Title
for name_string in data['Name']:
    data['Title'] = data['Name'].str.extract('([A-Za-z]+)\.', expand=True)
    
# Replacing rare titles 
mapping = {'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs', 'Major': 'Other', 
           'Col': 'Other', 'Dr' : 'Other', 'Rev' : 'Other', 'Capt': 'Other', 
           'Jonkheer': 'Royal', 'Sir': 'Royal', 'Lady': 'Royal', 
           'Don': 'Royal', 'Countess': 'Royal', 'Dona': 'Royal'}
           
data.replace({'Title': mapping}, inplace=True)
titles = ['Miss', 'Mr', 'Mrs', 'Royal', 'Other', 'Master']

# Replacing missing age by median/title 
for title in titles:
    age_to_impute = data.groupby('Title')['Age'].median()[titles.index(title)]
    data.loc[(data['Age'].isnull()) & (data['Title'] == title), 'Age'] = age_to_impute
    
# New feature : Family_size
data['Family_Size'] = data['Parch'] + data['SibSp'] + 1
data.loc[:,'FsizeD'] = 'Alone'
data.loc[(data['Family_Size'] > 1),'FsizeD'] = 'Small'
data.loc[(data['Family_Size'] > 4),'FsizeD'] = 'Big'

# Replacing missing Fare by median/Pclass 
fa = data[data["Pclass"] == 3]
data['Fare'].fillna(fa['Fare'].median(), inplace = True)

#  New feature : Child
data.loc[:,'Child'] = 1
data.loc[(data['Age'] >= 18),'Child'] =0

# New feature : Family Survival (https://www.kaggle.com/konstantinmasich/titanic-0-82-0-83)
data['Last_Name'] = data['Name'].apply(lambda x: str.split(x, ",")[0])
DEFAULT_SURVIVAL_VALUE = 0.5

data['Family_Survival'] = DEFAULT_SURVIVAL_VALUE
for grp, grp_df in data[['Survived','Name', 'Last_Name', 'Fare', 'Ticket', 'PassengerId',
                           'SibSp', 'Parch', 'Age', 'Cabin']].groupby(['Last_Name', 'Fare']):
                               
    if (len(grp_df) != 1):
        # A Family group is found.
        for ind, row in grp_df.iterrows():
            smax = grp_df.drop(ind)['Survived'].max()
            smin = grp_df.drop(ind)['Survived'].min()
            passID = row['PassengerId']
            if (smax == 1.0):
                data.loc[data['PassengerId'] == passID, 'Family_Survival'] = 1
            elif (smin == 0.0):
                data.loc[data['PassengerId'] == passID, 'Family_Survival'] = 0
                
for _, grp_df in data.groupby('Ticket'):
    if (len(grp_df) != 1):
        for ind, row in grp_df.iterrows():
            if (row['Family_Survival'] == 0) | (row['Family_Survival']== 0.5):
                smax = grp_df.drop(ind)['Survived'].max()
                smin = grp_df.drop(ind)['Survived'].min()
                passID = row['PassengerId']
                if (smax == 1.0):
                    data.loc[data['PassengerId'] == passID, 'Family_Survival'] = 1
                elif (smin == 0.0):
                    data.loc[data['PassengerId'] == passID, 'Family_Survival'] = 0
                    
####################################
# Encoding and pre-modeling
####################################                  

# dropping useless features
data = data.drop(columns = ['Age','Cabin','Embarked','Name','Last_Name',
                            'Parch', 'SibSp','Ticket', 'Family_Size'])

# Encoding features
target_col = ["Survived"]
id_dataset = ["Type"]
cat_cols   = data.nunique()[data.nunique() < 12].keys().tolist()
cat_cols   = [x for x in cat_cols ]
# numerical columns
num_cols   = [x for x in data.columns if x not in cat_cols + target_col + id_dataset]
# Binary columns with 2 values
bin_cols   = data.nunique()[data.nunique() == 2].keys().tolist()
# Columns more than 2 values
multi_cols = [i for i in cat_cols if i not in bin_cols]
# Label encoding Binary columns
le = LabelEncoder()
for i in bin_cols :
    data[i] = le.fit_transform(data[i])
# Duplicating columns for multi value columns
data = pd.get_dummies(data = data,columns = multi_cols )
# Scaling Numerical columns
std = StandardScaler()
scaled = std.fit_transform(data[num_cols])
scaled = pd.DataFrame(scaled,columns = num_cols)
# dropping original values merging scaled values for numerical columns
df_data_og = data.copy()
data = data.drop(columns = num_cols,axis = 1)
data = data.merge(scaled,left_index = True,right_index = True,how = "left")
data = data.drop(columns = ['PassengerId'],axis = 1)

# Target = 1st column
cols = data.columns.tolist()
cols.insert(0, cols.pop(cols.index('Survived')))
data = data.reindex(columns= cols)

# Cutting train and test
train = data[data['Type'] == 1].drop(columns = ['Type'])
test = data[data['Type'] == 0].drop(columns = ['Type'])

# X and Y
X_train = train.iloc[:, 1:20].values
y_train = train.iloc[:,0].values

####################################
# Keras - Neural Networks
####################################

# baseline model
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim = 18, activation = 'relu'))
    model.add(Dropout(0.2))
    model.add(Dense(8, activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

estimator = KerasClassifier(build_fn = create_baseline, epochs = 20, batch_size = 10, verbose = 1)
# kfold = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = False)
kfold = StratifiedKFold(n_splits = 5, shuffle = False)
results = cross_val_score(estimator, X_train, y_train, cv = kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

# X Test
X_test = test.iloc[:, 1:20].values

estimator.fit(X_train, y_train, epochs = 20, batch_size = 10)

# Predicting y_test
prediction = estimator.predict(X_test).tolist()

# List to series
data_check =  pd.read_csv("https://raw.githubusercontent.com/SalatielBairros/kaggle-titanic/main/data/original/test.csv")
se = pd.Series(prediction)
# Creating new column of predictions in data_check dataframe
data_check['check'] = se
data_check['check'] = data_check['check'].str.get(0)

series = []
for val in data_check.check:
    if val >= 0.5:
        series.append(1)
    else:
        series.append(0)
data_check['final'] = series

match = 0
nomatch = 0
for val in data_check.values:
    if val[1] == val[3]:
        match = match +1
    else:
        nomatch = nomatch +1

####################################
# Submission
#################################### 

temp = pd.DataFrame(pd.read_csv("https://raw.githubusercontent.com/SalatielBairros/kaggle-titanic/main/data/original/test.csv")['PassengerId'])
temp['Survived'] = data_check['final']

predictions = data_check['final'].tolist()
gt = pd.read_csv('../../data/original/ground_truth.csv')
print(classification_report(gt['Survived'], predictions))
print(accuracy_score(gt['Survived'], predictions))

temp.to_csv("../../data/submissions/keras_ex_nn.csv", index = False)


C:\Users\salat\AppData\Local\Temp/ipykernel_78732/2040922932.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = train.append(test)


Epoch 1/20


C:\Users\salat\AppData\Local\Temp/ipykernel_78732/2040922932.py:171: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn = create_baseline, epochs = 20, batch_size = 10, verbose = 1)


72/72 [==============================] - 0s 1ms/step - loss: 0.6880 - accuracy: 0.6152
Epoch 2/20
72/72 [==============================] - 0s 1ms/step - loss: 0.6122 - accuracy: 0.6461
Epoch 3/20
72/72 [==============================] - 0s 914us/step - loss: 0.5552 - accuracy: 0.7107
Epoch 4/20
72/72 [==============================] - 0s 1ms/step - loss: 0.5314 - accuracy: 0.7809
Epoch 5/20
72/72 [==============================] - 0s 971us/step - loss: 0.4946 - accuracy: 0.7879
Epoch 6/20
72/72 [==============================] - 0s 916us/step - loss: 0.4815 - accuracy: 0.8048
Epoch 7/20
72/72 [==============================] - 0s 873us/step - loss: 0.4586 - accuracy: 0.8160
Epoch 8/20
72/72 [==============================] - 0s 837us/step - loss: 0.4512 - accuracy: 0.8118
Epoch 9/20
72/72 [==============================] - 0s 903us/step - loss: 0.4335 - accuracy: 0.8301
Epoch 10/20
72/72 [==============================] - 0s 805us/step - loss: 0.4271 - accuracy: 0.8413
Epoch 11/20
72/7

In [204]:
data

,Survived,Sex,Type,Child,Pclass_1,Pclass_2,Pclass_3,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Other,Title_Royal,FsizeD_Alone,FsizeD_Big,FsizeD_Small,Family_Survival_0.0,Family_Survival_0.5,Family_Survival_1.0,Fare
0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,-0.503176
0,2,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,-0.503176
1,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0.734809
1,2,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0.734809
2,1,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,-0.490126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,1,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,-0.392009
887,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,-0.063340
888,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,-0.189974
889,1,1,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,-0.063340
